In [1]:
import numpy as np
from hmm_utils import HMM
from params import *

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import random

#some other libraries
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from typing import List

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roinaveiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Perprocessing

In [2]:
data = pd.read_csv("data/ner.csv", encoding = "latin1")
data = data.fillna(method="ffill")
data = data.rename(columns={'Sentence #': 'sentence'})
data.head(5)

/var/folders/6t/g0m1dgb95tl74_1f_f518pjh0000gn/T/ipykernel_2893/2847341886.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


,sentence,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [3]:
def pre_processing(text_column):
    # lowercase all text in the column
    text_column = text_column.str.lower()

    # replacing numbers with NUM token
    text_column = text_column.str.replace(r'\d+', 'NUM')

    # removing stopwords
    stop_words = set(stopwords.words('english'))
    text_column = text_column.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    return text_column

data_pre_precessed = pre_processing(data.Word)
#creating new dataframe with preprocessed word as a column
data_processed = data
data_processed['Word'] = data_pre_precessed

#removing the rows where word is empty
data_processed = data_processed[(data_processed['Word'] != '') | (data_processed['Word'].isna())]

In [4]:
data_processed

,sentence,Word,POS,Tag
0,Sentence: 1,thousands,NNS,O
2,Sentence: 1,demonstrators,NNS,O
4,Sentence: 1,marched,VBN,O
6,Sentence: 1,london,NNP,B-geo
8,Sentence: 1,protest,VB,O
...,...,...,...,...
1048567,Sentence: 47959,indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048571,Sentence: 47959,responded,VBD,O


# Select most common words 

In [5]:
# Most common words
N_w = 300
common_words = data_processed['Word'].value_counts().sort_values(ascending=False)[:N_w].index
data_reduced = data_processed[data_processed['Word'].isin(common_words)]

tags = list(set(data_reduced.POS.values))  # Unique POS tags in the dataset
words = list(set(data_reduced.Word.values))  # Unique words in the dataset
len(tags), len(words)

tags = np.sort(tags)
tags

array(['$', ',', '.', ':', 'CC', 'CD', 'DT', 'IN', 'JJ', 'JJR', 'JJS',
       'LRB', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'POS', 'PRP', 'RB', 'RBR',
       'RRB', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', '``'], dtype='<U4')

In [6]:
# Convert words and tags into numbers
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}
id2tag = {i: t for i, t in enumerate(tags)}
len(tags), len(words)
id2word = {}
for key in word2id:
    id2word[word2id[key]] = key


In [7]:
tag2id

{'$': 0,
 ',': 1,
 '.': 2,
 ':': 3,
 'CC': 4,
 'CD': 5,
 'DT': 6,
 'IN': 7,
 'JJ': 8,
 'JJR': 9,
 'JJS': 10,
 'LRB': 11,
 'MD': 12,
 'NN': 13,
 'NNP': 14,
 'NNPS': 15,
 'NNS': 16,
 'POS': 17,
 'PRP': 18,
 'RB': 19,
 'RBR': 20,
 'RRB': 21,
 'VB': 22,
 'VBD': 23,
 'VBG': 24,
 'VBN': 25,
 'VBP': 26,
 'VBZ': 27,
 '``': 28}

In [53]:
attack_tag = {
    0  : 1,
    1  : 2,
    2  : 3,
    4  : 6,
    5  : 13,
    6  : 4,
    7  : 6,
    8  : 9,
    9  : 10,
    10 : 9,
    11 : 19,
    12 : 22,
    13 : 16,
    14 : 15,
    15 : 14,
    16 : 13,
    17 : 17,
    18 : 19,
    19 : 20,
    20 : 21,
    21 : 20,
    22 : 23,
    23 : 25,
    24 : 22,
    25 : 23,
    26 : 27,
    27 : 26,
    28 : 2 
}

attack_tag = {
    0  : 0,
    1  : 1,
    2  : 1,
    3  : 3,
    4  : 4,
    5  : 5,
    6  : 6,
    7  : 7,
    8  : 8,
    9  : 9,
    10 : 10,
    11 : 11,
    12 : 12,
    13 : 16,
    14 : 14,
    15 : 15,
    16 : 16,
    17 : 17,
    18 : 18,
    19 : 19,
    20 : 20,
    21 : 21,
    22 : 22,
    23 : 23,
    24 : 24,
    25 : 25,
    26 : 26,
    27 : 27,
    28 : 2 
}

def seq2word(X):
    l = []
    for i in range(len(X)):
        l.append(id2word[X[i]])
    return l

def attack_seq(X):
    l = []
    for i in range(len(X)):
        l.append(attack_tag[X[i]])
    return l

In [54]:
tag2id

{'$': 0,
 ',': 1,
 '.': 2,
 ':': 3,
 'CC': 4,
 'CD': 5,
 'DT': 6,
 'IN': 7,
 'JJ': 8,
 'JJR': 9,
 'JJS': 10,
 'LRB': 11,
 'MD': 12,
 'NN': 13,
 'NNP': 14,
 'NNPS': 15,
 'NNS': 16,
 'POS': 17,
 'PRP': 18,
 'RB': 19,
 'RBR': 20,
 'RRB': 21,
 'VB': 22,
 'VBD': 23,
 'VBG': 24,
 'VBN': 25,
 'VBP': 26,
 'VBZ': 27,
 '``': 28}

## Create HMM manually

In [10]:
count_tags = dict(data_reduced.POS.value_counts())  # Total number of POS tags in the dataset
# Now let's create the tags to words count
count_tags_to_words = data_reduced.groupby(['POS']).apply(
    lambda grp: grp.groupby('Word')['POS'].count().to_dict()).to_dict()
# We shall also collect the counts for the first tags in the sentence
count_init_tags = dict(data_reduced.groupby('sentence').first().POS.value_counts())

# Create a mapping that stores the frequency of transitions in tags to it's next tags
count_tags_to_next_tags = np.zeros((len(tags), len(tags)), dtype=int)
sentences = list(data_reduced.sentence)
pos = list(data_reduced.POS)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        count_tags_to_next_tags[prevtagid][nexttagid] += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 329086/329086 [00:00<00:00, 2449353.68it/s]


In [11]:
startprob = np.zeros((len(tags),))
transmat = np.zeros((len(tags), len(tags)))
emissionprob = np.zeros((len(tags), len(words)))
num_sentences = sum(count_init_tags.values())
sum_tags_to_next_tags = np.sum(count_tags_to_next_tags, axis=1)
for tag, tagid in tqdm(tag2id.items(), position=0, leave=True):
    floatCountTag = float(count_tags.get(tag, 0))
    startprob[tagid] = count_init_tags.get(tag, 0) / num_sentences
    for word, wordid in word2id.items():
        emissionprob[tagid][wordid] = count_tags_to_words.get(tag, {}).get(word, 0) / floatCountTag
    for tag2, tagid2 in tag2id.items():
        transmat[tagid][tagid2] = count_tags_to_next_tags[tagid][tagid2] / sum_tags_to_next_tags[tagid]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 7704.75it/s]


# Build HMM

Include cutoff for probabilities equal to 0

In [12]:
cutoff = 0.001
startprob = startprob +  cutoff
startprob = startprob/ np.sum(startprob)
##
transmat =  transmat + cutoff
transmat = transmat / np.sum(transmat, axis=1)
##
emissionprob =  emissionprob + cutoff
emissionprob = emissionprob / np.sum(emissionprob, axis=1).reshape(-1,1)

In [13]:
hmm_n = HMM(len(tags), len(words))
hmm_n.startprob_ = startprob
hmm_n.transmat_ = transmat
hmm_n.emissionprob_ = emissionprob

# Attack HMM - APS

In [55]:
from solvers.aps_gibbs_class import aps_gibbs
from attackers.decoding_attacker import dec_attacker

In [56]:
T  = 53
n_obs = 300
n_hidden = 29
w1 = 3.0
w2 = 10.0 
k_value = 1000000.0
cool = np.arange(500,501, 1)
seq = 19*np.ones(T).astype(int)
X = np.zeros( (1, len(data_reduced.Word.values[:T])) )
for i in range(len(data_reduced.Word.values[:T])):
    X[0, i] = word2id[data_reduced.Word.values[i]]

###
X = np.zeros( (1, len(data_reduced.Word.values[:T])) )
for i in range(len(data_reduced.Word.values[:T])):
    X[0, i] = word2id[data_reduced.Word.values[i]]
    
A =  X.astype(int)

_, y_pred = hmm_n.nu(A[0])
y_pred

y = np.zeros(len(data_reduced.POS[:T]))
for i in range(len(data_reduced.POS[:T])):
    y[i] = tag2id[data_reduced.POS[:T].iloc[i]]
###

A =  X.astype(int)


In [84]:
rho_probs = np.ones(n_obs)
att = dec_attacker(hmm_n.startprob_ , hmm_n.transmat_, hmm_n.emissionprob_, rho_probs,
         A.T, w1, w2, seq, k_value)

In [ ]:
find_sol = aps_gibbs(att, cool, burnin=0.1, verbose=True)
sol, samples = find_sol.iterate(simulation_seconds=None)

Percentage completed: 0.0
Current state [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [ ]:
attack_obs = att.attack_X(np.ones_like(sol), sol)
attack_obs = attack_obs.squeeze().astype(int)
V, seq = hmm_n.nu(attack_obs)
probs, opt_s = hmm_n.nu(seq)

In [ ]:
opt_s

In [ ]:
np.argmax(sol, axis=1)

In [ ]:
A[0] == np.argmax(sol, axis=1)

In [48]:
#word2id

# Attack HMM - RS

In [57]:
from solvers.nn_RS.nn_RS import nn_RS
from attackers.decoding_attacker import dec_attacker

In [59]:
T  = 53
n_obs = 300
n_hidden = 29
w1 = 1.0
w2 = 5.0 
k_value = 1000000.0
X = np.zeros( (1, len(data_reduced.Word.values[:T])) )
for i in range(len(data_reduced.Word.values[:T])):
    X[0, i] = word2id[data_reduced.Word.values[i]]

###
X = np.zeros( (1, len(data_reduced.Word.values[:T])) )
for i in range(len(data_reduced.Word.values[:T])):
    X[0, i] = word2id[data_reduced.Word.values[i]]
    
A =  X.astype(int)

_, y_pred = hmm_n.nu(A[0])
y_pred

y = np.zeros(len(data_reduced.POS[:T]))
for i in range(len(data_reduced.POS[:T])):
    y[i] = tag2id[data_reduced.POS[:T].iloc[i]]
###

seq = attack_seq(y_pred.astype(int))


In [34]:
rho_probs = np.ones(n_obs)
att = dec_attacker(hmm_n.startprob_ , hmm_n.transmat_, hmm_n.emissionprob_, rho_probs,
         A.T, w1, w2, seq, k_value)

In [35]:
find_sol = nn_RS(att, "SA", RS_iters=5000, mcts_iters=10, sa_iters=10, eps=0.05, lr=0.005, verbose=True)
sol, samples = find_sol.iterate(simulation_seconds=None)

Percentage completed: 0.0
Best value: 
0.0
Percentage completed: 1.0
Best value: 
65.30698711240669
Percentage completed: 2.0
Best value: 
65.63731080021249
Percentage completed: 3.0
Best value: 
65.63731080021249
Percentage completed: 4.0
Best value: 
67.64330557021547
Percentage completed: 5.0
Best value: 
69.3771545601238
Percentage completed: 6.0
Best value: 
69.3771545601238
Percentage completed: 7.0
Best value: 
69.3771545601238
Percentage completed: 8.0
Best value: 
69.3771545601238
Percentage completed: 9.0
Best value: 
69.3771545601238
Percentage completed: 10.0
Best value: 
69.3771545601238
Percentage completed: 11.0
Best value: 
74.47054862319553
Percentage completed: 12.0
Best value: 
74.47054862319553
Percentage completed: 13.0
Best value: 
74.47054862319553
Percentage completed: 14.0
Best value: 
78.87990932180062
Percentage completed: 15.0
Best value: 
78.87990932180062
Percentage completed: 16.0
Best value: 
78.87990932180062
Percentage completed: 17.0
Best value: 
78.8

In [45]:
sol = find_sol.policy(eps=0.0, iters=500)

In [46]:
attack_obs = att.attack_X(np.ones_like(sol), sol)
attack_obs = attack_obs.squeeze().astype(int)
V, seq = hmm_n.nu(attack_obs)
probs, opt_s = hmm_n.nu(seq)

In [47]:
opt_s

array([14, 13,  8,  8, 14, 13, 14, 13, 14, 13,  8, 13,  8, 14, 13, 13, 14,
       13, 13, 14, 14, 13, 14, 13,  8,  5, 13, 14, 13,  8, 13,  8,  8, 14,
       13, 14, 13, 14,  8, 22, 13, 13,  8,  5,  8, 13,  8, 13, 14,  8,  8,
       13, 13])

In [48]:
np.mean(opt_s == seq)

0.22641509433962265

In [52]:
y_pred

array([16, 13, 14,  8, 16, 13,  2, 28,  2, 28, 14, 13,  5,  2, 28, 28,  2,
       28, 13,  2, 28, 13,  2, 28, 17, 13,  8,  2, 28, 14, 17, 14,  8, 16,
       13,  2, 28, 23,  1, 24,  1,  1,  2, 28, 13, 13,  8, 13, 16, 14, 14,
       17,  2])

In [50]:
y

array([16., 13., 14.,  8., 16., 13.,  2., 16., 25., 28., 14., 13.,  5.,
       13., 28., 28.,  2., 28., 13.,  2., 16., 13.,  2., 14., 17., 14.,
        8.,  2., 13., 14., 17., 14.,  8., 16., 13.,  2., 13., 23.,  1.,
       24.,  1.,  1.,  2., 14., 14., 14.,  8., 13., 16., 14., 14., 17.,
        2.])

In [51]:
seq

array([16, 13, 14,  8, 16, 13,  2, 28, 23, 13, 14, 13,  5,  2, 28, 28,  2,
       28, 28,  2, 16, 26,  2, 28,  8, 20, 13,  2, 28, 14, 17, 14, 14, 16,
       13,  2, 28, 23,  8, 24,  1,  1, 14, 15, 22, 13,  8, 13, 16, 14, 14,
       17,  1])

In [40]:
print( np.mean(opt_s == y) )
print( np.mean(y_pred == y) )

0.22641509433962265
0.7924528301886793


In [41]:
print( np.mean( attack_obs == A[0] ) )

0.6981132075471698


In [42]:
att.expected_utility(sol)

254.9316126197505

In [43]:
print(' '.join(seq2word(attack_obs))) 

thousands war iraq northern troops country . power killed number bush number one terrorist " " official " confirmed . police say . britain eastern earlier take . party britain 's iraq defense troops energy . march came northern including , , united states visit agency second day talks wednesday iran 's ,


In [44]:
print(' '.join(seq2word(A[0]))) 

thousands war iraq british troops country . soldiers killed " bush number one terrorist " " . " parliament . police number . britain 's party southern . party britain 's iraq british troops country . march came , including , , . international energy agency second day talks wednesday iran 's .
